In [3]:
from py2neo import Graph,NodeMatcher,Relationship,Node
import re
# 链接图数据库
graph = Graph("http://172.24.4.101:7474", auth=("neo4j", "ad_neo4j"))

In [4]:
import os
import math
import pandas as pd

def pd_nan(x):
    if isinstance(x, (int, float, complex)) and math.isnan(x):
        return False
    else:
        return True
def get_ex(xlb):
    ex = ''
    for i in xlb:
        if pd_nan(i):
            ex +=str(i)+' '
    return ex

In [5]:
graph.delete_all()

In [6]:
# tb = Node(relat[1],**{relat[1]+'_id':i1,relat[1]+'_name':name,'explain':description})
c = Node('c', name='东华dt')
cc1 = Node('c1', name='1.基础数据字典表')
cc2 = Node('c1', name='2.基础业务表')

graph.create(c)
graph.create(cc1)
graph.create(cc2)
graph.create(Relationship(c,cc1))
graph.create(Relationship(c,cc2))

In [7]:

with open('datas/dh_dt.txt', 'r') as fb:
    dt = fb.read()
dt = dt.split('\n')

path = 'datas/jxpdf_data4/'
path2 = 'datas/jxpdf_data2/'
lbs = os.listdir(path)
lbs22 = os.listdir(path2)
lbs2 = [re.split(r'[,：\s]+', i)[-1][:-4] for i in lbs22]
lbs = [i[:-4] for i in lbs]

In [117]:
lbs22[lbs2.index('INC_Itm')]

'3.1.2.11 库存项 INC_Itm.csv'

In [8]:
fz = cc1
for i in dt:
    nm = str(i).split()[0]
    names = ','.join(str(i).split()[1:])
    
    ll = len(nm.split('.'))
    cx = Node('c'+str(ll), name=nm+names)
    graph.create(cx)
    
    if i in lbs:
        df = pd.read_csv(path+i+'.csv')
        cy=None
        for v in df.values:
            ex = get_ex(v[4:])
            if pd_nan(v[2]) and ex:
                cy = Node('table',table_name=v[2],description=ex)
            elif pd_nan(v[2]):
                cy = Node('table',table_name=v[2])
            if cy:
                graph.create(cy)
                graph.create(Relationship(cx,cy))
                if str(v[2]) in lbs2:
                    df2 = pd.read_csv(path2 + lbs22[lbs2.index(v[2])])
                    cy2 = None
                    for i2 in df2.values:
                        ex = get_ex(i2[3:])
                        if pd_nan(i2[2]) and ex:
                            cy2 = Node('field',field_name=i2[2],description=ex)
                        elif pd_nan(i2[2]):
                            cy2 = Node('field',field_name=i2[2])
                        if cy2:
                            graph.create(cy2)
                            graph.create(Relationship(cy,cy2))
                            cy2 = None
                    
    bj = '.'.join(nm.split('.')[:-1])
    cq = 'MATCH (n:{}) WHERE n.name STARTS WITH "{}" RETURN n'.format('c'+str(ll-1),bj)
    for i in graph.run(cq):
        fz = i['n']
    if fz:
        graph.create(Relationship(fz,cx))
    else:
        print(bj)

In [63]:
with open('datas/dh_dt2.txt', 'r') as fb:
    dt = fb.read()
dt = dt.split('\n')

In [64]:
path = 'datas/jxpdf_data2/'
lbs = os.listdir(path)

In [65]:

fz = None
for i in dt:
    nm = str(i).split()[0]
    names = ' '.join(str(i).split()[1:])
    
    # print(nm)
    # print(names)
    ll = len(nm.split('.'))
    cx = Node('b'+str(ll), name=nm+names)
    graph.create(cx)
    
    names = re.split(r'[：\s]+', names)
    if len(names) >1:
        for i in lbs:
            if names[-1] in i:
                df = pd.read_csv(path+i)
                cy = None
                for i in df.values:
                    ex = get_ex(i[3:])
                    if pd_nan(i[2]) and ex:
                        cy = Node('field',field_name=i[2],description=ex)
                    elif pd_nan(i[2]):
                        cy = Node('field',table_name=i[2])
                    if cy:
                        graph.create(cy)
                        graph.create(Relationship(cx,cy))
                        cy = None
                    
    if fz:
        bj = '.'.join(nm.split('.')[:-1])
        cq = 'MATCH (n:{}) WHERE n.name STARTS WITH "{}" RETURN n'.format('b'+str(ll-1),bj)
        for i in graph.run(cq):
            graph.create(Relationship(i['n'],cx))
            fz = i['n']
    else:
        fz = cx

In [52]:
c = Node('c', name='东华dt')
cc1 = Node('c1', name='1.基础数据字典表')
cc2 = Node('c1', name='2.基础业务表')

graph.create(c)
graph.create(cc1)
graph.create(cc2)
graph.create(Relationship(c,cc1))
graph.create(Relationship(c,cc2))

[1 nan 'AccPDF_RowID' nan nan nan nan nan nan]
[2 1.0 'AccPDF_Date' '结算日期' 'Date' nan nan nan nan]
[3 2.0 'AccPDF_Time' '结算时间' 'Time' nan nan nan nan]
[4 3.0 'AccPDF_User_DR' '结算员用户\nRowID' 'DR' nan nan nan 'SS_User']
[5 4.0 'AccPDF_LastDate' '上次结算日期' 'Date' nan nan nan nan]
[6 5.0 'AccPDF_LastTime' '上次结算时间' 'Time' nan nan nan nan]
[7 6.0 'AccPDF_OperUser_DR' '收款员用户\nRowID' 'DR' nan nan nan 'SS_User']
[8 7.0 'AccPDF_INSFootUser' '财务接收人员' 'DR' nan nan nan 'SS_User']
[9 8.0 'AccPDF_INSFootDate' '财务接收日期' 'Date' nan nan nan nan]
[10 9.0 'AccPDF_INSFootTime' '财务接收时间' 'Time' nan nan nan nan]
[11 10.0 'AccPDF_Note1' nan nan nan nan nan nan]
[12 11.0 'AccPDF_Note2' nan nan nan nan nan nan]
[13 12.0 'AccPDF_Note3' nan nan nan nan nan nan]
[14 13.0 'AccPDF_Note4' nan nan nan nan nan nan]
[15 14.0 'AccPDF_footnum' nan nan nan nan nan nan]
[16 15.0 'AccPDF_refundnum' nan nan nan nan nan nan]
[17 16.0 'AccPDF_footsum' nan nan nan nan nan nan]
[18 17.0 'AccPDF_pdsum' nan nan nan nan nan nan]
[19 18.

In [56]:
i
get_ex(i[2:])

'AccPDF_Other9'

In [162]:
with open('ty.txt','w', encoding='utf-8') as f:
    for i in dt:
        i = re.split(r'[：\s]+', i)
        if len(i) >2:
            # f.write(i[-1])
            # f.write('\n')
            print(i[-2:])

['医院表', 'CT_Hospital']
['用户表', 'SS_User']
['科室表', 'CT_Loc']
['病区表', 'PAC_Ward']
['医护人员表', 'CT_CareProv']
['患者(身份)类型', 'CT_SocialStatus']
['患者费别', 'PAC_AdmReason']
['性别表', 'CT_Sex']
['邮编表', 'CT_Zip']
['出生地表、城市表', 'CT_City']
['婚姻状况表', 'CT_Marital']
['职业表', 'CT_Occupation']
['病人血型表', 'PAC_BloodType']
['教育水平表', 'CT_Education']
['出诊级别', 'RBC_SessionType']
['预约方式', 'RBC_AppointMethod']
['医嘱大类', 'OEC_OrderCategory']
['医嘱子类', 'ARC_ItemCat']
['医嘱项', 'ARC_ItmMast']
['频次表', 'PHC_Freq']
['用药途径(方式)', 'PHC_Instruc']
['药学项主表', 'PHC_DrgMast']
['药(理)学大类', 'PHC_Cat']
['药(理)学子类', 'PHC_SubCat']
['药学形态', 'PHC_DrgForm']
['管制分类', 'PHC_Poison']
['剂型', 'PHC_Form']
['库存项', 'INC_Itm']
['收费项目', 'DHC_TarItem']
['会计子分类', 'DHC_TarAcctCate']
['核算大类', 'DHC_TarAC']
['核算子分类', 'DHC_TarEMCCate']
['核算大类', 'DHC_TarEC']
['住院子分类', 'DHC_TarInpatCate']
['住院大类', 'DHC_TarIC']
['病案子分类', 'DHC_TarMRCate']
['病案大类', 'DHC_TarMC']
['门诊子分类', 'DHC_TarOutpatCate']
['门诊大类', 'DHC_TarOC']
['收费子分类', 'DHC_TarSubCate']
['收费大类', 'DHC_TarCate']
['

In [5]:
graph.nodes.match('table').all()
# graph.nodes.match('field',table_name='MZBL').first()

[Node('table', explain='服务器__包__网站__', table_id=408, table_name='ServerPackageSite'),
 Node('table', explain='患者__医保__信任__', table_id=409, table_name='PatYBCredence'),
 Node('table', explain='药材__模板__数据__', table_id=410, table_name='HerbonTemplateData'),
 Node('table', explain='日历__', table_id=411, table_name='Calendar'),
 Node('table', explain='部门__种类__', table_id=412, table_name='DeptKind'),
 Node('table', explain='治疗__预约__模板__', table_id=413, table_name='TreatmentBookingTemplate'),
 Node('table', explain='设置__', table_id=414, table_name='Setting'),
 Node('table', explain='患者__医保__支付、缴费__总结，摘要__', table_id=415, table_name='PatYBPaySummary'),
 Node('table', explain='未知____第三方__参与方__', table_id=416, table_name='VW_ThirdParty'),
 Node('table', explain='家庭__床位__', table_id=417, table_name='HomeBed'),
 Node('table', explain='部门__父母__', table_id=418, table_name='DeptParents'),
 Node('table', explain='医保__信任__种类__', table_id=419, table_name='YBCredenceKind'),
 Node('table', explain='设置__类别_

In [14]:
# 定义参数
parameters = {"x": "OrdersTemp"}

# 执行包含参数的 Cypher 语句
result = graph.run("MATCH (_:table) WHERE _.table_name = {x} RETURN _", parameters=parameters)

# 获得结果
for record in result:
    print(record)

Node('table', explain='医嘱__临时__', table_id=0, table_name='OrdersTemp')


In [5]:
for i in range(len(nd)):
    print(list(nd)[i]['class_name'])

其他身份信息
检查检验数据
护理数据
病程数据
手术数据
输血数据
出院数据
科研相关
患者身份信息
员工身份信息
病历数据
病人回访数据
用药数据
麻醉数据
住院数据
日常相关
物资相关
财务相关
科室信息
医学词汇
医疗交易信息
培训相关
后勤相关
人力资源相关
医院信息
保险信息
病史信息
体检信息


In [3]:
for i in list(nd):
    graph.delete(i)

NameError: name 'nd' is not defined

In [35]:
nn = Node("bgclass",bgclass_id=99,bgclass_name='东华HIS数据字典')
graph.create(nn)

In [6]:
graph.delete_all()

In [12]:
if graph.nodes.match('content',content_name="社保卡"):
    print(graph.nodes.match('content',content_name="社保卡").first())
    print(1)
else:
    print(0)

(_4713:content {content_id: 16, content_name: '\u793e\u4fdd\u5361', explain: '\u793e\u4fdd\u5361'})
1


In [40]:
print(len(list(graph.nodes.match('field'))))

962


In [9]:
with open('json/db.json','r')as fp:
    data = json.load(fp)

for i in data['library_0']:
    if i not in ['name','description']:
         data['library_0'][i]['name']


{'name': 'HIS',
 'description': '管理系统',
 'table_560': {'name': 'MZBL',
  'description': '门诊病历表',
  'field_0': {'name': 'TIME', 'description': '就诊日期'},
  'field_1': {'name': 'CASEALLERY', 'description': '过敏史'},
  'field_2': {'name': 'CASEMAIN', 'description': '主诉'},
  'field_3': {'name': 'CASENOW', 'description': '现病史'},
  'field_4': {'name': 'CASEOLD', 'description': '既往史'},
  'field_5': {'name': 'CHECKBODY', 'description': '查体'},
  'field_6': {'name': 'DIAGNOSE', 'description': '诊断'},
  'field_7': {'name': 'DOCNO', 'description': '医师代码'},
  'field_8': {'name': 'SEX', 'description': '性别'},
  'field_9': {'name': 'ID', 'description': '门诊号码'},
  'field_10': {'name': 'BIRTHDAY', 'description': '出生日期'},
  'field_11': {'name': 'AGE', 'description': '年龄'},
  'field_12': {'name': 'NAME', 'description': '名称'},
  'field_13': {'name': 'OPER_CODE', 'description': '操作员'},
  'field_14': {'name': 'OPER_DATE', 'description': '操作日期'},
  'field_15': {'name': 'REGDEPT', 'description': '科室'}},
 'table_359

In [11]:
matcher = NodeMatcher(graph=graph)

In [30]:
for i in data:
    print(i,data[i])
    a = matcher.match("table",table_name=i).first()
    print("第一个",a)
    b = matcher.match("class",class_name=data[i]).first()
    print("第二个",b)
    aaa = Relationship(a,"belong_to",b)
    graph.create(aaa)

MZBL 病历数据
第一个 (_9999:table {explain: '\u95e8\u8bca\u75c5\u5386\u8868', table_id: 0, table_name: 'MZBL'})
第二个 (_5320:class {class_id: 5, class_name: '\u75c5\u5386\u6570\u636e', explain: ''})
JYYZ 住院数据
第一个 (_9405:table {explain: '\u4f4f\u9662\u533b\u5631\u8868', table_id: 1, table_name: 'JYYZ'})
第二个 (_9358:class {class_id: 12, class_name: '\u4f4f\u9662\u6570\u636e', explain: ''})
BCJL 病程数据
第一个 (_9472:table {explain: '\u75c5\u7a0b\u8bb0\u5f55\u8868', table_id: 2, table_name: 'BCJL'})
第二个 (_9355:class {class_id: 6, class_name: '\u75c5\u7a0b\u6570\u636e', explain: ''})
SSJL 手术数据
第一个 (_9503:table {explain: '\u624b\u672f\u8bb0\u5f55\u8868', table_id: 3, table_name: 'SSJL'})
第二个 (_5322:class {class_id: 9, class_name: '\u624b\u672f\u6570\u636e', explain: ''})
SSGC 手术数据
第一个 (_9527:table {explain: '\u624b\u672f\u8fc7\u7a0b\u8868', table_id: 4, table_name: 'SSGC'})
第二个 (_5322:class {class_id: 9, class_name: '\u624b\u672f\u6570\u636e', explain: ''})
SSDJ 手术数据
第一个 (_9540:table {explain: '\u624b\u672

In [7]:
import json
with open('/home/tzy/Classification/sjzd.json','r') as fp:
    data = json.load(fp)
data

{'db_0': {'name': '��¥ҽԺ',
  'description': '��¥ҽԺ�����ֵ�',
  'table_0': {'name': 'AntiApproval1',
   'description': '����ҩ��������',
   'field_0': {'name': 'Idm', 'description': '����ҩ�����'},
   'field_1': {'name': 'DeptId', 'description': '���Ҵ���'},
   'field_2': {'name': 'StaffId', 'description': 'ҽ������'},
   'field_3': {'name': 'DeptKind',
    'description': '���ұ�־�����ٴ�1���Ű�:ҽ�\U000749a3�'},
   'field_4': {'name': 'NoticefieldFlag', 'description': '����֪ͨ��ʶ'}},
  'table_1': {'name': 'AntiDeptSetting',
   'description': '����ҩ���������',
   'field_5': {'name': 'DpetId', 'description': '���Ҵ���'},
   'field_6': {'name': 'Idm', 'description': '����ҩ�����'}},
  'table_2': {'name': 'AntiDoctorLevelSetting',
   'description': '����ҩ��ҽ����������',
   'field_7': {'name': 'AntiLevel', 'description': '������ҩƷ�ּ�'},
   'field_8': {'name': 'LowestApprovalDoctorLevel',
    'description': '�����������������÷��\u0ff9���ص����ҽ������'}},
  'table_3': {'name': 'AntiDoctorSetting',
   'des

In [11]:
for a,b in zip([1,2,3],[6,7,8]):
    print(a,b)


1 6
2 7
3 8


In [1]:
from similarities import Similarity
sim_model = Similarity(model_name_or_path = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",device='cpu')

/opt/miniconda3/envs/neo4j/lib/python3.8/site-packages/text2vec/utils/get_file.py:16: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-04-11 11:26:24.352 | DEBUG    | text2vec.sentence_model:__init__:74 - Use device: cpu


In [14]:
sim_model.similarity("血液成分明细","")

tensor([[0.6329]])

In [38]:
import time
for i in range(10000):
    st = time.time()
    cql = """MATCH (field:field)-[r]-() WITH field, count(r) as rel_count WHERE rel_count > 5 RETURN field.field_name"""
    print([x['field.field_name'] for x in graph.run(cql).data()])


['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', 'ClinicItemXh', 'Quantity', 'PatId']
['DeptId', 'Idm', 'StaffId', 'Name', '

In [6]:
import pandas as pd

In [74]:
df = pd.DataFrame(columns=['类别','相似度','关键字','原名称（表或字段名）'])
df

,类别,相似度,关键字,原名称（表或字段名）


In [75]:
dt = ('治疗数据', 0.8656, '治疗用法', '治疗__数据__')
dt[:3]+ ('sdsf',)

('治疗数据', 0.8656, '治疗用法', 'sdsf')

In [78]:
df.loc[7] = dt[:3]+ ('sdsf',)
df

,类别,相似度,关键字,原名称（表或字段名）
0,治疗数据,0.8656,治疗用法,sdsf
3,治疗数据,0.8656,治疗用法,sdsf
7,治疗数据,0.8656,治疗用法,sdsf


In [33]:
# 创建一个空的DataFrame
df = pd.DataFrame(columns=['A', 'B'])

# 逐行添加数据
for i in range(3):
    df.loc[i] = [i, i*2]

In [65]:
(1, 2, 3)+('a', 'b', 'c')

(1, 2, 3, 'a', 'b', 'c')

In [79]:
from config_1 import Config

In [80]:
cnf = Config()
cnf.mgkey_dic.keys()

dict_keys(['邮箱', '电话号码', 'qq', '微信', '密码', '身份证号码', '军官证号码', '居住证号码', '驾驶证号码', '社保卡', '护照号码', '银行', '银行号码', '信用卡号码', '支付宝', '地点', '姓名', '户口', '政治面貌', '生日', '性别', '职业', '婚姻状况', '种族', '学历', '港澳通行证', '公司', '税务号码', '牌照', '公积金', '密钥'])